In [1]:
import pandas as pd

# Load links from the file
links_df = pd.read_csv('data\wikispeedia_paths-and-graph\links.tsv', sep='\t', names=["current_article", "linked_article"], comment='#')

# Create a dictionary for faster lookup of links
link_map = {}
for idx, row in links_df.iterrows():
    link_map.setdefault(row['current_article'], []).append(row['linked_article'])


In [ ]:

from openai import OpenAI
api_key = "you_openai_api_key"

client = OpenAI(api_key = api_key)

def choose_next_article(current_article, possible_links, target_article, visited_articles, model):
    
    # Define the prompt
    prompt = f"""
    You are playing a game where you need to navigate from one Wikipedia article to the target article in the fewest steps possible.

    Current article: {current_article}
    Target article: {target_article}

    Articles you have previously visited: {', '.join(visited_articles) if visited_articles else "No article has been visited"}
    
    You can move to one of the following articles:
    {', '.join(possible_links)}

    Or if you think returning to the previous article is beneficial, you may do so. 
    
    Reply only with the name of the article you want to move to, with no explanation or marks.
    """

    # Make the updated API call
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert in navigating Wikipedia topics."},
            {"role": "user", "content": prompt}
        ]
    )

    # Return the chosen article and token usage
    return response.choices[0].message.content, response.usage.total_tokens


In [136]:
def play_wikispeedia_game(start_article, target_article, model="gpt-4"):
    current_article = start_article
    visited_articles = set()  # Track visited articles
    steps = 0
    total_tokens = 0

    while current_article != target_article:
        # Check if the article has outgoing links
        if current_article not in link_map:
            print(f"No outgoing links from {current_article}. Game over!")
            print(f"Total token usage: {total_tokens}")
            return

        # Get possible next articles
        possible_links = link_map[current_article]
        visited_articles_list = list(visited_articles)

        # Call GPT to get the next article
        next_article, token_usage = choose_next_article(current_article, possible_links, target_article, visited_articles_list, model=model)
        total_tokens += token_usage
        print(f"Step {steps}: Moved from '{current_article}' to '{next_article}'")

        # If next_article is the target, the game ends
        if next_article == target_article:
            print(f"Reached the target article '{target_article}' in {steps + 1} steps!")
            print(f"Total token usage: {total_tokens}")
            return

        # Add the current article to visited
        visited_articles.add(current_article)

        # Move to the next article
        current_article = next_article
        steps += 1

        # Limit steps to prevent infinite loops or excessive token usage
        if steps > 15:
            print("Step limit reached. Game terminated.")
            print(f"Total token usage: {total_tokens}")
            return

    # print(f"Reached the target article '{target_article}' in {steps} steps!")


In [137]:
start_article = "Aircraft"
target_article = "Google"

In [138]:
play_wikispeedia_game(start_article, target_article, model="gpt-3.5-turbo")


Step 0: Moved from 'Aircraft' to 'United_Kingdom'
Step 1: Moved from 'United_Kingdom' to 'William_Shakespeare'
Step 2: Moved from 'William_Shakespeare' to 'Ben_Jonson'
Step 3: Moved from 'Ben_Jonson' to 'William_Shakespeare'
Step 4: Moved from 'William_Shakespeare' to 'Actor'
Step 5: Moved from 'Actor' to 'William_Shakespeare'
Step 6: Moved from 'William_Shakespeare' to 'British_Empire'
Step 7: Moved from 'British_Empire' to 'Anglo-Saxons'
No outgoing links from Anglo-Saxons. Game over!
Total token usage: 2700


In [142]:
play_wikispeedia_game(start_article, target_article, model="gpt-4o")


Step 0: Moved from 'Aircraft' to 'Boeing_747'
Step 1: Moved from 'Boeing_747' to 'Seattle%2C_Washington'
Step 2: Moved from 'Seattle%2C_Washington' to 'Microsoft'
Step 3: Moved from 'Microsoft' to 'Bill_Gates'
Step 4: Moved from 'Bill_Gates' to 'Microsoft'
Step 5: Moved from 'Microsoft' to 'Computer'
Step 6: Moved from 'Computer' to 'Internet'
Step 7: Moved from 'Internet' to 'Cloud'
Step 8: Moved from 'Cloud' to 'Internet'
Step 9: Moved from 'Internet' to 'World_Wide_Web'
Step 10: Moved from 'World_Wide_Web' to 'Google'
Reached the target article 'Google' in 11 steps!
Total token usage: 2613


In [143]:
play_wikispeedia_game(start_article, target_article, model="gpt-4o-mini")


Step 0: Moved from 'Aircraft' to 'Boeing_747'
Step 1: Moved from 'Boeing_747' to 'United_States'
Step 2: Moved from 'United_States' to 'Computer'
Step 3: Moved from 'Computer' to 'Internet'
Step 4: Moved from 'Internet' to 'World_Wide_Web'
Step 5: Moved from 'World_Wide_Web' to 'History_of_the_Internet'
Step 6: Moved from 'History_of_the_Internet' to 'World_Wide_Web'
Step 7: Moved from 'World_Wide_Web' to 'Google'
Reached the target article 'Google' in 8 steps!
Total token usage: 3040
